# Работа со строковыми значениями

Материалы:
* Макрушин С.В. Лекция 8: Работа со строковыми значениям
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
* https://tproger.ru/translations/regular-expression-python/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {  
         'home_page': 'https://github.com/pypa/sampleproject',
         'keywords': 'sample setuptools development',
         'license': 'MIT'
      }

for k,v in obj.items():
    print(f"{k:10} = '{v}'")

home_page  = 'https://github.com/pypa/sampleproject'
keywords   = 'sample setuptools development'
license    = 'MIT'


2. Дана строка 'aaa--bbb==ccc__ddd'. Написать регулярное выражение для разбивки строки на список ['aaa','bbb','ccc','ddd'].

In [2]:
import re

#pattern = re.compile(r'[abcd]{3}')
pattern = re.compile(r'[a-z]{3}')
text = 'aaa--bbb==ccc__ddd'
pattern.findall(text)

['aaa', 'bbb', 'ccc', 'ddd']

3. Проверить корректность введенного E-mail

In [3]:
email = 'tv@gmail.com'

pattern = re.compile(r'\w+@[a-z]+\.[a-z]{2,3}$')
pattern.match(email).string

'tv@gmail.com'

4. Разбейте текст формулировки задачи 1 на слова.

In [4]:
import nltk
txt = """1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат 
таким образом, чтобы знак равенства оказался на одной и той же позиции во всех 
строках. Строковые литералы обернуть в кавычки."""

nltk.wordpunct_tokenize(txt)[:10]

['1', '.', 'Вывести', 'на', 'экран', 'данные', 'из', 'словаря', '`', 'obj']

## Лабораторная работа 8

### Форматирование строк

1.1 Загрузите данные из файла `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в таблицу `recipes` При помощи форматирования строк выведите информацию об id рецепта и кол-ве игредиентов 5 случайных рецептов в виде таблицы следующего вида:

    
    |    id     | n_in  |
    |-------------------|
    |  400894   |  13   |
    |   68588   |   8   |
    |  362081   |   6   |
    |   53408   |  12   |
    |  221203   |   4   |

In [5]:
import pandas as pd
import numpy as np

#заполнение выравнивание количество
recipes = pd.read_csv("./data/recipes_sample_with_tags_ingredients.csv")
recipes_sample = recipes.sample(5)
print(f"|{'id':^11}|{'n_in':^7}|\n|{'-'*19}|")
for row in recipes_sample.iterrows():
    print(f"|{row[1]['id']:^11}|{row[1]['n_ingredients']:^7}|")

|    id     | n_in  |
|-------------------|
|  124735   |  12   |
|  364530   |  14   |
|  266387   |   8   |
|  140892   |  10   |
|  194923   |   5   |


1.2 Напишите функцию `show_info`, которая для рецепта по его `id` создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название"

1. Шаг 1.
2. Шаг 2.
----------
#тэг1 #тэг2
```

    
Данные для создания строки получите из файлов `recipes_sample_with_tags_ingredients.csv`, `steps_sample.xml` (__ЛР4__) и `tags_sample.csv` (__ЛР5__). 
Выведите созданную строку на экран.

In [6]:
from bs4 import BeautifulSoup

tags_sample = pd.read_csv("./data/tags_sample.csv").set_index('id')
with open("./data/steps_sample.xml", "r") as file:
    data = BeautifulSoup(file)
steps_sample = {}
for recipe in data.find_all("recipe"):
    id = recipe.find("id")
    steps_sample[int(id.text)] = []
    for step in recipe.find_all("step"):
        steps_sample[int(id.text)].append(step.text)

In [21]:
def show_info(id=44123):
    name = recipes[recipes['id']==id]['name'].values[0]
    steps = steps_sample[id]
    tags = np.array(tags_sample.loc[44123]['tag'])
    print(f'\"{name}\"\n')
    for i in range(len(steps)):
        print(f'{i+1}. {steps[i]}.')
    print(f'-'*79)
    for tag in tags:
        print(f'#{tag} ')
    
show_info()

"george s at the cove  black bean soup"

1. in 1 / 4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes.
2. add thyme , oregano and basil.
3. saute 5 minutes more.
4. add wine and deglaze pan.
5. add hot chicken stock and reduce by one-third.
6. add worcestershire sauce , tabasco , smoked chicken , beans and broccoli florets.
7. simmer 5 minutes.
8. add cream , simmer 5 minutes more and season to taste.
9. drop in remaining butter , piece by piece , stirring until melted and serve immediately.
10. smoked chicken: on a covered grill , slightly smoke boneless chicken , cooking to medium rare.
11. chef meskan uses applewood chips and does not allow the grill to become too hot.
-------------------------------------------------------------------------------
#weeknight 
#time-to-make 
#course 
#main-ingredient 
#cuisine 
#preparation 
#occasion 
#north-american 
#soups-stews 
#beans 
#poultry 
#american 
#chicken 
#stove-top 
#dietary 
#gluten-free 
#comfort-food 
#

## Работа с регулярными выражениями

В задачах данного блока подразумевается, что вы не будете использовать никаких строковые методы (`split`, `startswith` и т.д.). Все задачи необходимо решить при помощи регулярных выражений.

2.1 Посчитайте кол-во отзывов, в которых встречаются числа.

In [8]:
reviews = pd.read_csv("./data/reviews_sample.csv")

p1 = re.compile(r'[0-9]')
k = 0
for review in reviews['review']:
    if bool(p1.search(str(review))):
        k += 1
print(k)

49246


2.2 Найдите все смайлики в отзывах. Смайлик состоит из трех частей: глаза (символ `=` или `:`), нос (символ `-`), губы (символ `)` или `(`). Смайлик может иметь вид "глаза-нос-губы" или "губы-нос-глаза". Нос может отсутствовать.

In [9]:
p2 = re.compile(r'[=:][-]?[()]|[()][-]?[=:]')
k = 0
for review in reviews['review']:
    if bool(p2.search(str(review))):
        print(p2.findall(str(review)), end="")
        k += 1

print(f'\nколичество -> {k}')

[':)'][':)', ':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)', ':)'][':)'][':)', ':)', ':)'][':)']['=)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':)', ':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)']['):'][':)']['=)'][':)'][':-)'][':('][':)'][':)'][':('][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)', ':)', ':)'][':-)'][':)'][':)'][':)'][':)']['=)'][':-)'][':)'][':)'][':)'][':)'][':-)', ':-)'][':)'][':)'][':)'][':)']['=)'][':)'][':)'][':)']['=)'][':)'][':)']['=)'][':)'][':)']['=)'][':)']['):']['=)'][':-)'][':)'][':)'][':)']['=)'][':)', ':)'][':)'][':)'][':)'][':('][':)'][':)'][':)'][':)'][':('][':)'][':)'][':('][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':-('][':)'][':)'][':)']['(:'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':

[':)'][':)'][':('][':)'][':)']['=)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)', ':)', ':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)']['=)'][':)'][':)'][':-)'][':)'][':)'][':)']['=)'][':)'][':)'][':-)']['=)'][':)'][':)'][':)'][':)'][':(', ':)'][':)'][':)']['):'][':-)'][':-)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)'][':)']['=)'][':-)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':-)'][':-)'][':)'][':-)'][':('][':)'][':)'][':)'][':-('][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':)'][':)'][':('][':)'][':)'][':-)'][':-)'][':('][':-)'][':)'][':)'][':)']['=)'][':)'][':)']['=)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':('][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':-)'][':)'][':-('][':-)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)']['

[':)'][':)'][':)'][':)'][':-)', ':-)'][':)', ':)'][':)'][':-)'][':)'][':-)'][':)'][':)'][':)'][':)'][':-)'][':-)'][':)'][':)'][':)'][':)'][':)'][':-(', ':-)'][':)'][':-)'][':-)'][':)'][':)'][':)'][':)', ':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':-)'][':)'][':)'][':('][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':)', ':)'][':)'][':-('][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':)'][':(', ':('][':)'][':)'][':)'][':)']['(:'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)', ':-)', ':-)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)']['=)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':-)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)'][':('][':-)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][

[':)'][':-)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':-)'][':-)'][':)', ':)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':-)'][':-)'][':-)'][':('][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)', ':)'][':)']['=)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':-)'][':-)'][':)'][':)']['=)'][':('][':)'][':-('][':)'][':)'][':)'][':)'][':)'][':('][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)']['(:'][':)'][':)'][':)'][':)'][':(']['=('][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)']['=)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':-)'][':-)'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':-)'][':-)'][':)'][':)'][':)'][':)'][':('][':)'][':)'][':)'][':)']['=)'][

[':)'][':)'][':)', ':)'][':)'][':-)'][':)'][':)'][':)'][':)']['=)'][':)'][':)'][':-)'][':-)'][':-)'][':)', ':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)']['=)']['=)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':('][':('][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)', ':)', ':)'][':)'][':)'][':-)'][':-)'][':-)'][':)'][':)', ':)'][':)'][':)']['=)'][':)'][':('][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':)'][':('][':)'][':)'][':(', ':)'][':)', ':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)', ':-)'][':)'][':)'][':)'][':)'][':-)'][':('][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':-)'][':)'][':-)'][':)

[':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)'][':)'][':)'][':-)'][':)'][':-)'][':('][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':('][':-)'][':)'][':-)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)', ':)'][':)'][':-)'][':)'][':)'][':-('][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)']['(-:'][':('][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)'][':)']['=)'][':)'][':)'][':)'][':-)'][':)'][':)'][':)'][':)'][':('][':-)'][':)'][':)'][':)']['(:', '(:'][':)'][':)'][':)', ':)'][':)'][':)', ':)'][':-)']['=)'][':)']['=)'][':)'][':)'][':)'][':)'][':)']['):'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':)'][':-('][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)', ':)', ':)'][':-)'][':-)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':)'][':-)'][':-)'][':)'][':)'][':)'][':)'][':('][':)'][':)']['=)'][':)'][':-)'][':)'][':)'][':)']['=)'][':)', ':)'][':)'][':)']['=)'][':)'][':)']['

2.3 Проверьте, что все даты в датасете имеют вид "YYYY-MM-DD". Продемонстрируйте работу вашего решения, приведя пример из датасета и контрпример не из датасета.

In [10]:
p3 = re.compile(r'^([12][09][0-9][0-9])\-([0][1-9]|1[012])\-([0-2][0-9]|[3][01]$)')
k = 0
for date in reviews['date']:
    if bool(p3.search(str(date))):
        k += 1
if k == len(reviews['date']):
    print('все даты корректны')

dates = ['2003/05-01', '2003-05-01', 'a2003-05-01', '2003-75-01', '3003-05-01']
print([date for date in dates if re.match(p3, date)])

все даты корректны
['2003-05-01']


2.4 Используя строку-результат задачи 1.2, найдите первое слово каждого шага в рецепте

2.5 Используя регулярные выражения, удалите из описаний все символы, кроме английских букв, цифр и пробелов. Сохраните предобработанные описания в файл `preprocessed_descriptions.csv`, содержащий 2 столбца: `name` и `preprocessed_descriptions`.

In [11]:
#\s -пробелы, \d - цифры
recipes_names_desc = recipes[["name", "description"]]
p5 = r'[^a-zA-Z\s\d]+'
for i in range(recipes_names_desc.shape[0]):    
    recipes_names_desc.loc[i]['description'] = re.sub(p5, '', str(recipes_names_desc.loc[i]['description']))
recipes_names_desc.to_csv('preprocessed_descriptions.csv')
recipes_names_desc

,name,description
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,these were so go it surprised even me
3,italian gut busters,my sisterinlaw made these for us at a family g...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...
...,...,...
29995,zurie s holey rustic olive and cheddar bread,this is based on a french recipe but i changed...
29996,zwetschgenkuchen bavarian plum cake,this is a traditional fresh plum cake thought ...
29997,zwiebelkuchen southwest german onion cake,this is a traditional late summer early fall s...
29998,zydeco soup,this is a delicious soup that i originally fou...


### Сегментация текста

3.1 Разбейте отзывы из файла `recipes_sample.csv` (__ЛР2__) на предложения, а предложения - на слова (используйте `sent_tokenize` и `word_tokenize` из `nltk`). Каждый отзыв представьте в виде списка списков: внешний список - предложения, вложенные списки - слова в предложении. Исключите знаки препинания из списков слов.

`'Предложение номер один. Предложение номер два.' => [['Предложение', 'номер', 'один'], ['Предложение', 'номер', 'два']]`

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tatiana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
#mb from reviews_sample
sentences_words = []
for review in reviews["review"].fillna(""):
    for sentence in nltk.sent_tokenize(review):
        sentences_words.append(nltk.word_tokenize(sentence))
        
sentences_words

[['Last',
  'week',
  'whole',
  'sides',
  'of',
  'frozen',
  'salmon',
  'fillet',
  'was',
  'on',
  'sale',
  'in',
  'my',
  'local',
  'supermarket',
  ',',
  'so',
  'I',
  'bought',
  'tons',
  '(',
  'okay',
  ',',
  'only',
  '3',
  ',',
  'but',
  'total',
  'weight',
  'was',
  'over',
  '10',
  'pounds',
  ')',
  '.'],
 ['This',
  'recipe',
  'is',
  'perfect',
  'for',
  'salmon',
  'fillet',
  ',',
  'even',
  'though',
  'it',
  'calls',
  'for',
  'salmon',
  'steaks',
  '.'],
 ['I',
  'cut',
  'up',
  'the',
  'salmon',
  'into',
  'individual',
  'portions',
  'and',
  'followed',
  'the',
  'instructions',
  'exactly',
  '.'],
 ['I',
  "'m",
  'on',
  'one',
  'of',
  'those',
  'food',
  'combining',
  'diets',
  ',',
  'so',
  'I',
  'left',
  'out',
  'the',
  'white',
  'wine',
  'but',
  'added',
  'just',
  'a',
  'dash',
  'of',
  'white',
  'wine',
  'vinegar',
  'instead',
  '(',
  'just',
  'a',
  'little',
  'bit',
  ',',
  'not',
  'enough',
  'to',
  '

3.2 Посчитайте кол-во уникальных слов среди всех отзывов (без учета регистра и знаков препинания).

In [15]:
without_punct = set()
pattern = re.compile(r'[a-zA-Z]')
for sentence in sentences_words:
    for word in sentence:
        if pattern.match(word):
            without_punct.add(word.lower())
            
len(without_punct)

57784

In [16]:
list(without_punct)[:5]

['ur', 'warm-', 'hommus', 'jokingly', 'daydream']

3.3 Найдите 5 самых длинных (по количеству слов; без учета знаков препинания) отзывов в датасете и выведите их в порядке убывания длины.

In [17]:
review_len = {}
for review in reviews["review"].fillna(""):
    words = nltk.word_tokenize(review)
    review_len[review] = len(words)
    
review_len

{"Last week whole sides of frozen salmon fillet was on sale in my local supermarket, so I bought tons (okay, only 3, but total weight was over 10 pounds).  This recipe is perfect for salmon fillet, even though it calls for salmon steaks.  I cut up the salmon into individual portions and followed the instructions exactly.  I'm on one of those food combining diets, so I left out the white wine but added just a dash of white wine vinegar instead (just a little bit, not enough to change the taste of the dish).  Super yummy, and leftovers for lunch today (lucky me)!": 122,
 "So simple and so tasty!  I used a yellow capsicum in place of the green because that's what I had on hand.  This came together so fast.  Perfect meal if you don't have a lot of time.  Easy, healthy and tasty.  Thanks Stardustannie!  Made for PAC Fall 2007.": 58,
 'Very nice breakfast HH, easy to make and yummy with fresh hot coffe.  Instead of toast I served on recipe #97694 by Paula G that I had made earlier and turned

In [18]:
for k in sorted(review_len, key=review_len.get, reverse=True)[:5]:
    print(review_len[k])

1214
1089
811
787
764


3.4 Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на любом предложении из отзывов.


In [19]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Tatiana\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [20]:
def what_part_of_speech(sentence):
    words, parts = "", ""
    for word, part in nltk.pos_tag(nltk.word_tokenize(sentence)):
        width = max(len(word), len(part))
        words += f"{word:^{width}} "
        parts += f"{part:^{width}} "
        w = max(len(word), len(part))
    print(parts)
    print(words)
    
what_part_of_speech(reviews["review"].iloc[5])

PRP  VBD   TO VB   JJ   NN  CC  PRP  RB    VBD     RB    JJ  .  NNS    NNP   . 
 I  forgot to add skim milk but it  still tasted pretty good ! Thanks Dancer ! 
